In [59]:
from tifffile import imread
import napari
import pandas as pd
import numpy as np
import dask.array as da

In [2]:
im_path = r'R:\CellDive\SH178-145-8\SH178-145-8_Final\SH178-145-8_1.0.4_R000_DAPI__FINAL_F.ome.tif'
df_path = r'R:\data_analysis\Wayne\Dormancy\20240220-1811_aivia_allslides.csv'

In [3]:
store = imread(im_path,aszarr=True)

l = []
# as Napari seems to open only 6 levels anyway 
for i in range(6):
    d = da.from_zarr(store, i)
    l.append(d)

In [4]:
viewer = napari.Viewer()
viewer.add_image(l,contrast_limits=[0,10000])

<Image layer 'l' at 0x188e6d88a30>

In [5]:
df = pd.read_csv(df_path)

In [47]:
df_sel = df[df.slide == 'SH178-145-8']
len(df_sel)

541713

In [48]:
df_sel

,Center of Mass X (µm),Center of Mass Y (µm),Outline Area (µm²),Pixel-based Area (µm²),Perimeter (µm),Length (µm),Breadth (µm),Aspect Ratio,Length/Breadth Ratio,Circularity,...,Center of Mass Y (px),Outline Area (px²),Pixel-based Area (px²),Direct Line Length (px),Interval Length (px),Area Overlapped (px²),Nearest Object Distance (px),Mean Object Distance 3 (px),Mean Object Distance 5 (px),Nearest Objects Count (px)
81279,5363.758789,1617.814697,751.5,824.0,158.325902,64.287979,19.340187,0.267606,0.905464,0.376734,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81280,5546.299805,1620.538452,578.0,644.0,145.740115,49.454281,22.215218,0.527273,0.899169,0.341963,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81281,5497.528809,1632.540405,2260.5,2348.0,199.923882,64.113068,48.421593,0.753846,0.986355,0.710698,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81282,5713.237305,1626.337524,426.5,468.0,95.497475,28.037973,22.941374,0.727273,0.849636,0.587686,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81283,5555.106934,1632.491455,895.5,955.0,143.095454,55.047771,22.835400,0.686275,1.079368,0.549571,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622987,NaN,NaN,273.5,302.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
622988,NaN,NaN,310.0,340.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
622989,NaN,NaN,369.5,403.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
622990,NaN,NaN,274.0,302.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_coords = df_sel[['Center of Mass X (µm)','Center of Mass Y (µm)']]

In [37]:
# skip rows with NaN values
df_coords = df_coords.dropna()
len(df_coords)

137543

In [45]:
# swap the x and y coordinates
df_coords['row'] = df_coords['Center of Mass Y (µm)'] + 200
df_coords['col'] = df_coords['Center of Mass X (µm)'] + 5430  

In [46]:
viewer.add_points(df_coords[['row','col']].values, size=10, face_color='red', edge_color='black')

<Points layer 'Points' at 0x18a0196fd90>

In [52]:
from matplotlib.path import Path

In [54]:
# Function to check if a point is inside the polygon
def is_point_inside_polygon(x, y, polygon_path):
    
    return polygon_path.contains_point((x, y))

In [81]:
df_coords['selected'] = False

for vertices in viewer.layers['Shapes'].data:

    polygon_path = Path(np.concatenate([vertices,vertices[0,:][None]]))
    
    is_inside = df_coords.apply(lambda row: is_point_inside_polygon(row['row'], row['col'], polygon_path), axis=1)
    print(np.sum(is_inside))

    df_coords['selected'] = df_coords['selected'] | is_inside


1025
348


In [82]:
viewer.add_points(df_coords.loc[df_coords.selected,['row','col']].values, size=10, face_color='yellow', edge_color='black')

<Points layer 'Points [1]' at 0x18a1aa716a0>

In [60]:
np.sum(df_coords['selected'])

0

In [61]:
len(df_coords)

137543